In [1]:
import pandas as pd
import numpy as np
import pylab as pp
import os
import requests
import io
import urllib.request

os.chdir("D:\Stocks\PersianCal")
import PersianCal

In [2]:
os.chdir('D:/Stocks/')
file = 'NAMES.csv'
codes = pd.read_csv(file)
# code = str(codes.Codes[code_number])

In [26]:
stock_name = 'ولساپا'
code_number = codes.Names.tolist().index(stock_name)
code = str(codes.Codes[code_number])

# code = '70474983732269112'

url = "http://www.tsetmc.com/tsev2/data/Export-txt.aspx?t=i&a=1&b=0&i=" + code
r = requests.get(url, allow_redirects=True).content

# open('7769065543658313.csv', 'wb').write(r.content);

df = pd.read_csv(io.StringIO(r.decode('utf-8')))

In [27]:
## Datetime conversion
df.rename(columns={"<DTYYYYMMDD>": "Date"}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')

# Dropping any NaNs
df.dropna(inplace=True)

In [28]:
df['Date'] =pd.to_datetime(df.Date)
df = df.sort_values(by=['Date'])

In [29]:
df

,<TICKER>,Date,<FIRST>,<HIGH>,<LOW>,<CLOSE>,<VALUE>,<VOL>,<OPENINT>,<PER>,<OPEN>,<LAST>
3151,S*Rayan.Saipa,2005-07-11,5000.0,5000.0,5000.0,5000.0,120000000000,24000000,1136,D,5000.0,5000.0
3150,S*Rayan.Saipa,2005-07-26,4161.0,4400.0,4132.0,4250.0,740723368,176733,48,D,5000.0,4250.0
3149,S*Rayan.Saipa,2005-07-27,4250.0,4462.0,4201.0,4385.0,854115473,194752,80,D,4250.0,4385.0
3148,S*Rayan.Saipa,2005-07-30,4604.0,4604.0,4604.0,4604.0,587659164,127641,38,D,4385.0,4604.0
3147,S*Rayan.Saipa,2005-07-31,4834.0,4834.0,4604.0,4792.0,36292977645,7686093,488,D,4604.0,4792.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4,S*Rayan.Saipa,2020-07-18,8660.0,9450.0,8660.0,8960.0,2789760134570,311200476,40800,D,9110.0,9230.0
3,S*Rayan.Saipa,2020-07-19,9350.0,9350.0,8520.0,8880.0,2318575429210,261151839,37460,D,8960.0,8670.0
2,S*Rayan.Saipa,2020-07-20,9000.0,9320.0,8850.0,9170.0,1702816817330,185628868,27636,D,8880.0,8950.0
1,S*Rayan.Saipa,2020-07-21,9170.0,9170.0,8720.0,8770.0,1238074306870,141187239,21176,D,9170.0,8720.0


In [6]:
d = []
for i in range(len(df["Date"])):
    d.append(PersianCal.Christian(str(df["Date"][i])[0:10]).persian_string()) #Tabdil tarikh be Irani

In [7]:
for i in range(len(df["Date"])):
    df["Date"] = df["Date"].replace(df["Date"][i], d[i])

In [8]:
date_list = []
for date in df["Date"]:
    date_list.append(date[:7])
    
Dates = df["Date"].tolist() #kole tarikh ha
myDates = list(dict.fromkeys(date_list)) #Eshterake Tatikh ha

In [9]:
ext = np.zeros((len(myDates), 2)) #[[max], [min]]
ind = np.zeros((len(myDates), 1)).tolist()

#first indicator
for date in Dates:
    if (date[:7] in myDates[0]):
        ind1 = Dates.index(date) #یافتن مکان آخرین روز ماه

ind[0] = ind1

# First max and min:

In [10]:
a = [[], [], [], []]
# names = ["<Open>", "<High>", "<Low>", "<Close>"]
names = ["<FIRST>", "<HIGH>", "<LOW>", "<CLOSE>"]

for i in range(4):
    for j in names:
        a[i] = df[j][0:ind1+1].tolist()

ext[0][0] = max(max(a))
ext[0][1] = min(min(a))

# Calculating all of the extremums:

In [11]:
for i in range(1, len(myDates)):
    for date in Dates:
        if (date[:7] in myDates[i]):
            ind1 = Dates.index(date)
    ind[i] = ind1
    a = [[], [], [], []]
    for ii in range(4):
        for j in names:
            a[ii] = df[j][ind[i-1]+1:ind[i]+1].tolist()
    ext[i][0] = max(max(a))
    ext[i][1] = min(min(a))

In [12]:
# import pylab as pp

# %matplotlib notebook
# import pylab as pp

# pp.plot(DataF["Date"], DataF["Max"], color='skyblue', label = "Max")
# pp.plot(DataF["Date"], DataF["Min"], color='red', label = "Min")
# pp.legend();

In [13]:
def getIndexes(dfObj, value):
    ''' Get index positions of value in dataframe i.e. dfObj.'''
    listOfPos = list()
    # Get bool dataframe with True at positions where the given value exists
    result = dfObj.isin([value])
    # Get list of columns that contains the value
    seriesObj = result.any()
    columnNames = list(seriesObj[seriesObj == True].index)
    # Iterate over list of columns and fetch the rows indexes where value exists
    for col in columnNames:
        rows = list(result[col][result[col] == True].index)
        for row in rows:
            listOfPos.append((row, col))
    # Return a list of tuples indicating the positions of value in the dataframe
    return listOfPos

In [14]:
# listOfPositions = getIndexes(df[ind[4]+1:ind[5]+1], ext[5][1])
# print('Index positions of 1835000 in Dataframe: ')
# for i in range(len(listOfPositions)):
#     print('Position ', i, ' (Row index , Column Name) : ', listOfPositions[i])

In [15]:
minDate = []
firstPos = getIndexes(df[0:ind[0]+1], ext[0][1])
minDate.append(df.Date[firstPos[0][0]])

for j in range(len(ext)-1):
    listOfPositions = getIndexes(df[ind[j]+1:ind[j+1]+1], ext[j+1][1])
    for i in range(len(listOfPositions)):
        minDate.append(df.Date[listOfPositions[0][0]])

minDate = list(dict.fromkeys(minDate))

In [16]:
maxDate = []
firstPos2 = getIndexes(df[0:ind[0]+1], ext[0][0])
maxDate.append(df.Date[firstPos2[0][0]])

for j in range(len(ext)-1):
    listOfPositions = getIndexes(df[ind[j]+1:ind[j+1]+1], ext[j+1][0])
    for i in range(len(listOfPositions)):
        maxDate.append(df.Date[listOfPositions[0][0]])
        
maxDate = list(dict.fromkeys(maxDate))

In [17]:
first = []
last = []

for i in range(len(maxDate)):
    if (int(maxDate[i][8:10]) < int(minDate[i][8:10])):
        first.append(ext[i, 0])
        last.append(ext[i, 1])
    else:
        first.append(ext[i, 1])
        last.append(ext[i, 0])

In [18]:
# res = {"Date": myDates,
#        "First": first,
#        "Last": last}

res = {"Date": myDates,
       "First": first,
       "Last": last,
       "Max": ext[:, 0],
       "Min": ext[:, 1]}

resF = pd.DataFrame(res)

In [19]:
#making empty dates
myDates2 = []
for i in range(2*len(myDates)):
    myDates2.append(0)

#Filling Dates with 1 and 15
for i in range(len(myDates)):
    myDates2[2*i] = myDates[i] + '-01'
    myDates2[2*i+1] = myDates[i] + '-15' 

In [20]:
#Making empty final data
final_data = []
for i in range(len(myDates2)):
    final_data.append(0)
    
#Filling final data
for i in range(len(myDates)):
    final_data[2*i] = first[i]
    final_data[2*i+1] = last[i]

In [21]:
result_data = {"Date": myDates2,
               "Extrema": final_data}
result_dataF = pd.DataFrame(result_data)

# result_dataF.set_index('Date', inplace=True)
# result_dataF.to_csv(filename + '_output' + '.csv')

In [22]:
d2 = []
for i in range(len(result_dataF["Date"])):
    d2.append(PersianCal.Persian
             (str(result_dataF["Date"][i])[0:10]).christian_string2()) #Tabdil tarikh be Irani
    
    
for i in range(len(result_dataF["Date"])):
    result_dataF["Date"] = result_dataF["Date"].replace(result_dataF["Date"][i], d2[i])

In [23]:
result_data2 = {"Date": result_dataF.Date,
                "Open": result_dataF.Extrema,
                "High": result_dataF.Extrema,
                "Low": result_dataF.Extrema,
                "Close": result_dataF.Extrema}
result_data2F = pd.DataFrame(result_data2)

In [24]:
os.chdir("D:/")

file_name = str(df['<TICKER>'][1]).replace('*', '')

result_data2F.set_index('Date', inplace=True)
result_data2F.to_csv(file_name + '_output' + '.csv')

In [25]:
df

,<TICKER>,Date,<FIRST>,<HIGH>,<LOW>,<CLOSE>,<VALUE>,<VOL>,<OPENINT>,<PER>,<OPEN>,<LAST>
3151,S*Rayan.Saipa,1384-04-20,5000.0,5000.0,5000.0,5000.0,120000000000,24000000,1136,D,5000.0,5000.0
3150,S*Rayan.Saipa,1384-05-04,4161.0,4400.0,4132.0,4250.0,740723368,176733,48,D,5000.0,4250.0
3149,S*Rayan.Saipa,1384-05-05,4250.0,4462.0,4201.0,4385.0,854115473,194752,80,D,4250.0,4385.0
3148,S*Rayan.Saipa,1384-05-08,4604.0,4604.0,4604.0,4604.0,587659164,127641,38,D,4385.0,4604.0
3147,S*Rayan.Saipa,1384-05-09,4834.0,4834.0,4604.0,4792.0,36292977645,7686093,488,D,4604.0,4792.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4,S*Rayan.Saipa,1399-04-28,8660.0,9450.0,8660.0,8960.0,2789760134570,311200476,40800,D,9110.0,9230.0
3,S*Rayan.Saipa,1399-04-29,9350.0,9350.0,8520.0,8880.0,2318575429210,261151839,37460,D,8960.0,8670.0
2,S*Rayan.Saipa,1399-04-30,9000.0,9320.0,8850.0,9170.0,1702816817330,185628868,27636,D,8880.0,8950.0
1,S*Rayan.Saipa,1399-04-31,9170.0,9170.0,8720.0,8770.0,1238074306870,141187239,21176,D,9170.0,8720.0
